In [1]:
%matplotlib inline

TorchText로 텍스트 분류하기
==================================
**번역**: `김강민 <https://github.com/gangsss>` , `김진현 <https://github.com/lewhe0>`

이 튜토리얼에서는 ``torchtext`` 에 포함되어 있는 텍스트 분류
데이터셋의 사용 방법을 살펴 봅니다. 데이터셋은 다음을 포함합니다.

   - AG_NEWS,
   - SogouNews,
   - DBpedia,
   - YelpReviewPolarity,
   - YelpReviewFull,
   - YahooAnswers,
   - AmazonReviewPolarity,
   - AmazonReviewFull

이 예제에서는 ``TextClassification`` 의 데이터셋들 중 하나를 이용해 분류를 위한
 지도 학습 알고리즘을 훈련하는 방법을 보여줍니다. 

ngrams를 이용하여 데이터 불러오기
-----------------------------------

Bag of ngrams 피쳐는 지역(local) 단어 순서에 대한 부분적인 정보를 포착하기 위해 적용합니다.
실제 상황에서는 bi-gram이나 tri-gram은 단 하나의 단어를 이용하는 것보다 더 많은 이익을 주기 때문에 적용됩니다.
예를 들면 다음과 같습니다.

   "load data with ngrams"
   Bi-grams 결과: "load data", "data with", "with ngrams"
   Tri-grams 결과: "load data with", "data with ngrams"

``TextClassification`` 데이터셋은 ngrams method을 지원합니다. ngrams을 2로 설정하면,
데이터셋 안의 예제 텍스트는 각각의(single) 단어들에 bi-grams 문자열이 더해진 리스트가 될 것입니다.

In [12]:
import torch
import torchtext
from torchtext.datasets import text_classification
import os

In [13]:
NGRAMS = 2
BATCH_SIZE = 16

In [2]:
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=NGRAMS, vocab=None)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ag_news_csv.tar.gz: 11.8MB [00:01, 10.0MB/s]
120000lines [00:05, 23340.97lines/s]
120000lines [00:11, 10897.91lines/s]
7600lines [00:00, 10768.21lines/s]


In [27]:
for x in train_dataset:
    print(x)
    break

(2, tensor([    572,     564,       2,    2326,   49106,     150,      88,       3,
           1143,      14,      32,      15,      32,      16,  443749,       4,
            572,     499,      17,      10,  741769,       7,  468770,       4,
             52,    7019,    1050,     442,       2,   14341,     673,  141447,
         326092,   55044,    7887,     411,    9870,  628642,      43,      44,
            144,     145,  299709,  443750,   51274,     703,   14312,      23,
        1111134,  741770,  411508,  468771,    3779,   86384,  135944,  371666,
           4052]))


모델 정의하기
-------------

우리의 모델은
`EmbeddingBag`레이어와 선형 레이어로 구성됩니다 (아래 그림 참고).
``nn.EmbeddingBag``는 임베딩들로 구성된 '가방'의 평균을 계산합니다.
이때 텍스트(text)의 각 원소는 그 길이가 다를 수 있습니다. 텍스트의
길이는 오프셋(offset)에 저장되어 있으므로 여기서 ``nn.EmbeddingBag``
에 패딩을 사용할 필요는 없습니다.

덧붙여서, ``nn.EmbeddingBag`` 은 임베딩의 평균을 즉시 계산하기 때문에,
텐서들의 시퀀스를 처리할 때 성능 및 메모리 효율성 측면에서의 장점도
갖고 있습니다.

![](../_static/img/text_sentiment_ngrams_model.png)





<b>torch.nn.EmbeddingBag</b>(num_embeddings: int, embedding_dim: int, max_norm: Optional\[float] = None, norm_type: float = 2.0, scale_grad_by_freq: bool = False, mode: str = 'mean', sparse: bool = False, _weight: Optional\[torch.Tensor] = None, include_last_offset: bool = False)<br>
Computes sums or means of ‘bags’ of embeddings, without instantiating the intermediate embeddings.

- num_embeddings (int) – size of the dictionary of embeddings
- embedding_dim (int) – the size of each embedding vector
- max_norm (float, optional) – If given, each embedding vector with norm larger than max_norm is renormalized to have norm max_norm.
- norm_type (float, optional) – The p of the p-norm to compute for the max_norm option. Default 2.
- scale_grad_by_freq (boolean, optional) – if given, this will scale gradients by the inverse of frequency of the words in the mini-batch. Default False. Note: this option is not supported when mode="max".
- mode (string, optional) – "sum", "mean" or "max". Specifies the way to reduce the bag. "sum" computes the weighted sum, taking per_sample_weights into consideration. "mean" computes the average of the values in the bag, "max" computes the max value over each bag. Default: "mean"
- sparse (bool, optional) – if True, gradient w.r.t. weight matrix will be a sparse tensor. See Notes for more details regarding sparse gradients. Note: this option is not supported when mode="max".
- include_last_offset (bool, optional) – if True, offsets has one additional element, where the last element is equivalent to the size of indices. This matches the CSR format. Note: this option is currently only supported when mode="sum".

<b>Inputs</b>: input (LongTensor), offsets (LongTensor, optional), and
per_index_weights (Tensor, optional)
- If input is 2D of shape (B, N),<br>
it will be treated as B bags (sequences) each of fixed length N, and this will return B values aggregated in a way depending on the mode. offsets is ignored and required to be None in this case.
- If input is 1D of shape (N),<br>
it will be treated as a concatenation of multiple bags (sequences). offsets is required to be a 1D tensor containing the starting index positions of each bag in input. Therefore, for offsets of shape (B), input will be viewed as having B bags. Empty bags (i.e., having 0-length) will have returned vectors filled by zeros.


offsets이 뭔지 모르겠음

In [68]:
# an Embedding module containing 10 tensors of size 3
embedding_sum = nn.EmbeddingBag(10, 3, mode='sum') 
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([0,1,2,4,5,4,3,2,9])
offsets = torch.LongTensor([0,6])
embedding_sum(input, offsets)

tensor([[ 1.1342, -3.8707, -0.8586],
        [ 3.5805, -3.0927,  0.6699]], grad_fn=<EmbeddingBagBackward>)

In [3]:
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

인스턴스 생성하기
-----------------

AG_NEWS 데이터셋에는 4 종류의 레이블이 달려 있으며, 따라서 클래스의 개수도 4개 입니다.

   1 : World (세계)
   2 : Sports (스포츠)
   3 : Business (경제)
   4 : Sci/Tec (과학/기술)

어휘집의 크기(Vocab size)는 어휘집(vocab)의 길이와 같습니다 (여기에는
각각의 단어와 ngrame이 모두 포함됩니다). 클래스의 개수는 레이블의 종류
수와 같으며, AG_NEWS의 경우에는 4개 입니다.




In [5]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

배치 생성을 위한 함수들
-----------------------




텍스트 원소의 길이가 다를 수 있으므로, 데이터 배치와 오프셋을 생성하기
위한 사용자 함수 generate_batch()를 사용하려 합니다. 이 함수는
``torch.utils.data.DataLoader`` 의 ``collate_fn`` 인자로 넘겨줍니다.

``collate_fn`` 의 입력은 그 크기가 batch_size인 텐서들의 리스트이며,
``collate_fn`` 은 이들을 미니배치로 묶는 역할을 합니다. 여러분이
주의해야 할 점은, ``collate_fn`` 를 선언할 때 최상위 레벨에서 정의해야
한다는 점입니다. 그래야 이 함수를 각각의 워커에서 사용할 수 있음이
보장됩니다.

원본 데이터 배치 입력의 텍스트 원소들은 리스트 형태이며, 이들을 하나의
텐서가 되도록 이어 붙인 것이 ``nn.EmbeddingBag`` 의 입력이 됩니다.
오프셋은 텍스트의 경계를 나타내는 텐서이며, 각 원소가 텍스트 텐서의
어느 인덱스에서 시작하는지를 나타냅니다. 레이블은 각 텍스트 원소의
레이블을 담고 있는 텐서입니다.




<b>collate_fn (callable, optional)</b>: merges a list of samples to form a
        mini-batch of Tensor(s).  Used when using batched loading from a
        map-style dataset.

In [6]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum은 dim 차원의 요소들의 누적 합계를 반환합니다.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

모델을 학습하고 결과를 평가하는 함수 정의하기
---------------------------------------------




PyTorch 사용자라면
`torch.utils.data.DataLoader`를 활용하는 것을 추천합니다. 또한 이를 사용하면 데이터를 쉽게 병렬적으로
읽어올 수 있습니다 (이에 대한 튜토리얼은 `이 문서 <https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html>`
를 참고하시기 바랍니다). 우리는 여기서 ``DataLoader`` 를 이용하여
AG_NEWS 데이터셋을 읽어오고, 이를 모델로 넘겨 학습과 검증을 진행합니다.




In [16]:
data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)

In [19]:
for x in data:
    print(x)
    break

(tensor([   262,   2729,  39488,  ...,  10847, 119565, 746074]), tensor([   0,   63,  264,  299,  382,  475,  648,  741,  828,  883,  976, 1051,
        1230, 1297, 1354, 1441]), tensor([3, 3, 2, 1, 3, 3, 3, 3, 0, 3, 1, 0, 0, 1, 2, 3]))


In [7]:
from torch.utils.data import DataLoader

In [8]:
def train_func(sub_train_):
    # Train the model
    # 모델을 학습합니다
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # 학습율을 조절합니다
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

데이터셋을 분할하고 모델 수행하기
---------------------------------

원본 AG_NEWS에는 검증용 데이터가 포함되어 있지 않기 때문에, 우리는 학습
데이터를 학습 및 검증 데이터로 분할하려 합니다. 이때 데이터를 분할하는
비율은 0.95(학습)와 0.05(검증) 입니다. 우리는 여기서 PyTorch의
핵심 라이브러리 중 하나인
`torch.utils.data.dataset.random_split`함수를 사용합니다.

`CrossEntropyLoss`기준(criterion)은 각 클래스에 대해 nn.LogSoftmax()와 nn.NLLLoss()를
합쳐 놓은 방식입니다.
`SGD` optimizer는 확률적 경사 하강법를 구현해놓은 것입니다. 처음의 학습율은
4.0으로 두었습니다. 매 에폭을 진행하면서 학습율을 조절할 때는
`StepLR`을 사용합니다.




In [78]:
import time
from torch.utils.data.dataset import random_split

In [80]:
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

RuntimeError: Tensor for argument #1 'weight' is on CPU, but expected it to be on GPU (while checking arguments for embedding_bag_cuda)

평가 데이터로 모델 평가하기
---------------------------




In [10]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0002(test)	|	Acc: 87.6%(test)


평가 데이터셋을 통한 결과를 확인합니다...



임의의 뉴스로 평가하기
----------------------

현재까지 구한 최고의 모델로 골프 뉴스를 테스트해보려 합니다. 레이블에
대한 정보는
`여기에 <https://pytorch.org/text/datasets.html?highlight=ag_news#torchtext.datasets.AG_NEWS>`__
나와 있습니다.




In [69]:
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

In [70]:
ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

In [75]:
for _ in ngrams_iterator('i am good person'.split(), 2):
    print(_)

i
am
good
person
i am
am good
good person


In [77]:
vocab['ago']

472

In [11]:
def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])

This is a Sports news
